In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [2]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# Convolutional neural network (two convolutional layers)
"""nn.Conv2d(in_channels, out_channels, kernel_size)"""
"""nn.BatchNorm2d(num_features): num_features is just number of input channels"""
"""nn.Linear(in_features, out_features)"""
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        # input shape: (None,1,28,28)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2), #output shape: (None,16,28,28)
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) #output shape: (None,16,14,14)
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), #output shape: (None,32,14,14)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) #output shape: (None,32,7,7)
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        # input shape: (None,1,28,28)
        out = self.layer1(x) #output shape: (None,16,14,14)
        out = self.layer2(out) #output shape: (None,32,7,7)
        """The reshape function also consider the 0th, i.e. batch dimension, different from Keras.Reshape!"""
        out = out.reshape(out.size(0), -1) #output shape: (None,32*7*7)
        out = self.fc(out) #output shape: (None,10)
        return out

model = ConvNet(num_classes).to(device)

In [5]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        """PyTorch is channel first, different from TensorFlow!"""
        """images.size(): torch.Size([100, 1, 28, 28])"""
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1330
Epoch [1/5], Step [200/600], Loss: 0.0593
Epoch [1/5], Step [300/600], Loss: 0.0385
Epoch [1/5], Step [400/600], Loss: 0.0894
Epoch [1/5], Step [500/600], Loss: 0.0807
Epoch [1/5], Step [600/600], Loss: 0.0940
Epoch [2/5], Step [100/600], Loss: 0.0412
Epoch [2/5], Step [200/600], Loss: 0.1250
Epoch [2/5], Step [300/600], Loss: 0.0341
Epoch [2/5], Step [400/600], Loss: 0.0742
Epoch [2/5], Step [500/600], Loss: 0.0187
Epoch [2/5], Step [600/600], Loss: 0.0347
Epoch [3/5], Step [100/600], Loss: 0.0379
Epoch [3/5], Step [200/600], Loss: 0.0600
Epoch [3/5], Step [300/600], Loss: 0.0139
Epoch [3/5], Step [400/600], Loss: 0.0507
Epoch [3/5], Step [500/600], Loss: 0.0237
Epoch [3/5], Step [600/600], Loss: 0.0635
Epoch [4/5], Step [100/600], Loss: 0.1111
Epoch [4/5], Step [200/600], Loss: 0.0170
Epoch [4/5], Step [300/600], Loss: 0.0440
Epoch [4/5], Step [400/600], Loss: 0.0123
Epoch [4/5], Step [500/600], Loss: 0.0031
Epoch [4/5], Step [600/600], Loss:

In [6]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
# has effect on specific layers like Dropout and BatchNorm
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        #Tensor.data: tensor
        _, predicted = torch.max(outputs.data, 1)
        #Tensor.size(): torch.Size, Tensor.size(0): int
        total += labels.size(0)
        #Tensor.item(): number
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


# nn.Module.state_dict(): collections.OrderedDict
"""
model.state_dict().keys(): odict_keys([
 #Conv2d
'layer1.0.weight', 'layer1.0.bias',
 #BatchNorm2d
'layer1.1.weight', 'layer1.1.bias', 'layer1.1.running_mean', 'layer1.1.running_var', 'layer1.1.num_batches_tracked',
 #Conv2d
'layer2.0.weight', 'layer2.0.bias', 
 #BatchNorm2d
 'layer2.1.weight', 'layer2.1.bias', 'layer2.1.running_mean', 'layer2.1.running_var', 'layer2.1.num_batches_tracked', 
 #Linear
 'fc.weight', 'fc.bias'])
"""
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.0 %
